In [1]:
# 1 - Import the necessary libraries load the data and start a new notebook. Using the same data as 
#the previous lab: we_fn_use_c_marketing_customer_value_analysis.csv

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats

In [3]:
data = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')
data.head(5)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
# 2 - Find all of the categorical data. Save it in a categorical_df variable.

In [5]:
categorical_df = data.select_dtypes(include = object)

In [6]:
categorical_df.head(3)

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize


In [7]:
#3 - Check for NaN values and decide what to do with them, do it now.

In [8]:
categorical_df.isna().sum()

Customer             0
State                0
Response             0
Coverage             0
Education            0
Effective To Date    0
EmploymentStatus     0
Gender               0
Location Code        0
Marital Status       0
Policy Type          0
Policy               0
Renew Offer Type     0
Sales Channel        0
Vehicle Class        0
Vehicle Size         0
dtype: int64

In [9]:
# 4 - Check all unique values of columns.

In [10]:
categorical_df.nunique()

Customer             9134
State                   5
Response                2
Coverage                3
Education               5
Effective To Date      59
EmploymentStatus        5
Gender                  2
Location Code           3
Marital Status          3
Policy Type             3
Policy                  9
Renew Offer Type        4
Sales Channel           4
Vehicle Class           6
Vehicle Size            3
dtype: int64

In [11]:
# 5 -Check dtypes. Do they all make sense as categorical data?

In [12]:
categorical_df.dtypes

Customer             object
State                object
Response             object
Coverage             object
Education            object
Effective To Date    object
EmploymentStatus     object
Gender               object
Location Code        object
Marital Status       object
Policy Type          object
Policy               object
Renew Offer Type     object
Sales Channel        object
Vehicle Class        object
Vehicle Size         object
dtype: object

In [13]:
#All the columns makes sense to be categorical except the column 'Effective to Date'. I will change the type to datetime.
categorical_df['Effective To Date']= pd.to_datetime(categorical_df['Effective To Date'])

In [14]:
categorical_df.dtypes

Customer                     object
State                        object
Response                     object
Coverage                     object
Education                    object
Effective To Date    datetime64[ns]
EmploymentStatus             object
Gender                       object
Location Code                object
Marital Status               object
Policy Type                  object
Policy                       object
Renew Offer Type             object
Sales Channel                object
Vehicle Class                object
Vehicle Size                 object
dtype: object

In [15]:
# 6 - Does any column contain alpha and numeric data? Decide how to clean it and do it now.

In [16]:
categorical_df.head(3)

#The column 'customer','policy' and 'Renew offer type' has alpha and numerical data

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size
0,BU79786,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize


In [17]:
categorical_df['customer_id'] = categorical_df['Customer'].str.extract('(\d+)')

In [18]:
categorical_df['Customer'] = categorical_df['Customer'].str.replace('\d+', '')

C:\Users\ines_\AppData\Local\Temp\ipykernel_12872\740417506.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  categorical_df['Customer'] = categorical_df['Customer'].str.replace('\d+', '')


In [19]:
categorical_df.head(3)

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size,customer_id
0,BU,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize,79786
1,QZ,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize,44356
2,AI,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize,49188


In [20]:
categorical_df['policy_number'] = categorical_df['Policy'].str.extract('(\d+)')

In [21]:
categorical_df['Policy'] = categorical_df['Policy'].str.replace('\d+', '')

C:\Users\ines_\AppData\Local\Temp\ipykernel_12872\3477090376.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  categorical_df['Policy'] = categorical_df['Policy'].str.replace('\d+', '')


In [22]:
categorical_df.head(3)

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size,customer_id,policy_number
0,BU,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Corporate L,Offer1,Agent,Two-Door Car,Medsize,79786,3
1,QZ,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Personal L,Offer3,Agent,Four-Door Car,Medsize,44356,3
2,AI,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Personal L,Offer1,Agent,Two-Door Car,Medsize,49188,3


In [23]:
categorical_df['Renew Offer Type'].unique()

array(['Offer1', 'Offer3', 'Offer2', 'Offer4'], dtype=object)

In [24]:
categorical_df['Renew Offer Type'] = categorical_df['Renew Offer Type'].str.extract('(\d+)')

In [25]:
categorical_df

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size,customer_id,policy_number
0,BU,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Corporate L,1,Agent,Two-Door Car,Medsize,79786,3
1,QZ,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Personal L,3,Agent,Four-Door Car,Medsize,44356,3
2,AI,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Personal L,1,Agent,Two-Door Car,Medsize,49188,3
3,WW,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L,1,Call Center,SUV,Medsize,63253,2
4,HB,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,Personal L,1,Agent,Four-Door Car,Medsize,64268,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA,California,No,Basic,Bachelor,2011-02-10,Employed,M,Urban,Married,Personal Auto,Personal L,2,Web,Four-Door Car,Medsize,72316,1
9130,PK,California,Yes,Extended,College,2011-02-12,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L,1,Branch,Four-Door Car,Medsize,87824,3
9131,TD,California,No,Extended,Bachelor,2011-02-06,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L,1,Branch,Four-Door Car,Medsize,14365,2
9132,UP,California,No,Extended,College,2011-02-03,Employed,M,Suburban,Married,Personal Auto,Personal L,3,Branch,Four-Door Car,Large,19263,2


In [26]:
# 7. Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions and do it now.

In [27]:
categorical_df.nunique()

Customer              677
State                   5
Response                2
Coverage                3
Education               5
Effective To Date      59
EmploymentStatus        5
Gender                  2
Location Code           3
Marital Status          3
Policy Type             3
Policy                  3
Renew Offer Type        4
Sales Channel           4
Vehicle Class           6
Vehicle Size            3
customer_id          8688
policy_number           3
dtype: int64

In [28]:
#drop the column costumer_id because it has a lot of unique values
categorical_df.drop(columns=["customer_id"],axis=1,inplace=True)
categorical_df

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size,policy_number
0,BU,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Corporate L,1,Agent,Two-Door Car,Medsize,3
1,QZ,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Personal L,3,Agent,Four-Door Car,Medsize,3
2,AI,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Personal L,1,Agent,Two-Door Car,Medsize,3
3,WW,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L,1,Call Center,SUV,Medsize,2
4,HB,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,Personal L,1,Agent,Four-Door Car,Medsize,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA,California,No,Basic,Bachelor,2011-02-10,Employed,M,Urban,Married,Personal Auto,Personal L,2,Web,Four-Door Car,Medsize,1
9130,PK,California,Yes,Extended,College,2011-02-12,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L,1,Branch,Four-Door Car,Medsize,3
9131,TD,California,No,Extended,Bachelor,2011-02-06,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L,1,Branch,Four-Door Car,Medsize,2
9132,UP,California,No,Extended,College,2011-02-03,Employed,M,Suburban,Married,Personal Auto,Personal L,3,Branch,Four-Door Car,Large,2


In [29]:
#8. Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?

In [30]:
categorical_df["Policy Type"].unique()

array(['Corporate Auto', 'Personal Auto', 'Special Auto'], dtype=object)

In [31]:
categorical_df["Policy"].unique()

array(['Corporate L', 'Personal L', 'Special L'], dtype=object)

In [32]:
#The information seems to be the same. Policy type says Corporate, Personal and Special auto. The column Policy also
#identifys Corporate, Personal and Special. I would delete one of this columns.

categorical_df.drop(['Policy','Customer'],axis=1,inplace=True)

In [33]:
#9 Check number of unique values in each column, can they be combined in any way to ease encoding? 
#Comment your thoughts and make those changes.

In [34]:
categorical_df.nunique()

State                 5
Response              2
Coverage              3
Education             5
Effective To Date    59
EmploymentStatus      5
Gender                2
Location Code         3
Marital Status        3
Policy Type           3
Renew Offer Type      4
Sales Channel         4
Vehicle Class         6
Vehicle Size          3
policy_number         3
dtype: int64

In [36]:
#The Customer column has 9134 unique values, which makes it hard to encode it without adding too many features. 
#So we can extract year to create new column. The rest of the columns have a small number of unique values, which means 
#they can be easily encoded using one-hot encoding.

categorical_df['Year'] = categorical_df['Effective To Date'].dt.year
categorical_df

,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size,policy_number,Year
0,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,1,Agent,Two-Door Car,Medsize,3,2011
1,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,3,Agent,Four-Door Car,Medsize,3,2011
2,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,1,Agent,Two-Door Car,Medsize,3,2011
3,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,1,Call Center,SUV,Medsize,2,2011
4,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,1,Agent,Four-Door Car,Medsize,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,California,No,Basic,Bachelor,2011-02-10,Employed,M,Urban,Married,Personal Auto,2,Web,Four-Door Car,Medsize,1,2011
9130,California,Yes,Extended,College,2011-02-12,Employed,F,Suburban,Divorced,Corporate Auto,1,Branch,Four-Door Car,Medsize,3,2011
9131,California,No,Extended,Bachelor,2011-02-06,Unemployed,M,Suburban,Single,Corporate Auto,1,Branch,Four-Door Car,Medsize,2,2011
9132,California,No,Extended,College,2011-02-03,Employed,M,Suburban,Married,Personal Auto,3,Branch,Four-Door Car,Large,2,2011


In [ ]:
#10 Save the cleaned catagorical dataframe as categorical.csv You will use this file again this week.

In [37]:
categorical_df.to_csv("categorical.csv")